[🥭 Entropy Markets](https://entropy.trade/) support is available at: [Docs](https://docs.entropy.trade/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/entropymarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

[![error](https://myerror.org/badge_logo.svg)](https://myerror.org/v2/gh/blockworks-foundation/entropy-explorer-examples/HEAD?labpath=ImmediateBuyAndSell.ipynb) [](https://myerror.org/v2/gh/blockworks-foundation/entropy-explorer-examples/HEAD?labpath=ImmediateBuyAndSell.ipynb) on error.

_🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

# 🥭 Immediate Buy and Sell

This notebook shows how to buy and sell from the current orderbook, rather than placing orders to be executed later.

This example will SELL 1 SOL on SOL/USDC and then BUY it back.

The code uses 'Immediate Or Cancel' (IOC) orders. Orders of this type never appear on the orderbook - they exist to explicitly 'take' orders from the orderbook. If it's not possible to fulfill the IOC order quantity at the specified price, it is cancelled.


In [ ]:
import decimal
import entropy


# Use this function to display balances because it always fetches fresh data
def show_balances(context, wallet, group):
    # Find the wrapped SOL token
    sol_token = context.instrument_lookup.find_by_symbol_or_raise("SOL")

    # Find the current SPL token balance
    token_account = entropy.TokenAccount.fetch_largest_for_owner_and_token(context, wallet.address, sol_token)
    print("Wrapped SOL in wallet:", token_account.value)

    # Find the SOL-specific details in the account
    accounts = entropy.Account.load_all_for_owner(context, wallet.address, group)
    for slot in accounts[0].slots:
        if slot.base_instrument == sol_token:
            print("SOL in account:       ", slot.net_value)
            print("USDC in account:      ", accounts[0].shared_quote.net_value)
            print()
            return


# Use our hard-coded mainnet wallet for DeekipCw5jz7UgQbtUbHQckTYGKXWaPQV4xY93DaiM6h.
# For real-world use you'd load the bytes from the environment or a file.
wallet = entropy.Wallet(bytes([181,213,227,47,41,229,109,138,15,82,26,7,230,184,88,102,197,215,238,155,136,196,138,92,98,154,67,68,47,140,90,40,248,149,223,193,241,51,4,196,126,32,211,66,90,137,249,160,132,246,38,29,88,16,252,116,12,83,117,158,40,98,178,54]))

# Create a 'mainnet' Context
with entropy.ContextBuilder.build(cluster_name="mainnet") as context:
    # Load the wallet's account
    group = entropy.Group.load(context)
    accounts = entropy.Account.load_all_for_owner(context, wallet.address, group)
    account = accounts[0]

    max_slippage = decimal.Decimal("0.005")  # Allow at most 0.5% slippage from top-of-book
    quantity = decimal.Decimal(1)

    print("Initial balances before trades:")
    show_balances(context, wallet, group)

    market_operations = entropy.operations(context, wallet, account, "SOL/USDC", dry_run=False)

    sell_signatures = market_operations.market_sell(quantity, max_slippage)
    print("IOC SELL transaction signatures:", sell_signatures)

    print("Waiting for sell transaction to confirm...\n")
    entropy.WebSocketTransactionMonitor.wait_for_all(
            context.client.cluster_ws_url, sell_signatures, commitment="processed"
        )

    show_balances(context, wallet, group)

    buy_signatures = market_operations.market_buy(quantity, max_slippage)
    print("IOC BUY transaction signatures:", buy_signatures)

    print("Waiting for buy transaction to confirm...\n")
    entropy.WebSocketTransactionMonitor.wait_for_all(
            context.client.cluster_ws_url, buy_signatures, commitment="processed"
        )

    show_balances(context, wallet, group)

print("Example complete.")